2021-06-24 베이즈정리 복습

# 베이즈 정리 
- 중간에 들어온 정보로 인해(사건 B가 발생) 사건 A 사전확률이 어떻게 사후확률로 변하는지 보여준다.



# P(A|B) = (P(B|A)P(A))/P(B)
- P(A) : 사전확률
- P(B|A) : 가능도
- P(B) : 정규화상수(크기조절, 별 의미 없음)

# 베이즈정리의 확장 
- 일반형 : 전체확률의 법칙 사용해서 정규화 상수 변형
- 축소형 : A와 A여집합 두 개를 전체확률의 법칙 적용, 정규화 상수 변형

In [2]:
# 조건부확률모형 구현
# 사전확률 모형
from pgmpy.factors.discrete import TabularCPD

cpd_X = TabularCPD('X',2,np.array([[1-0.002],[0.002]]))
print(cpd_X)

+------+-------+
| X(0) | 0.998 |
+------+-------+
| X(1) | 0.002 |
+------+-------+


In [3]:
# 가능도 모형 
cpd_Y_on_X = TabularCPD('Y',2,np.array([[0.95,0.01],[0.05,0.99]]),evidence=['X'],evidence_card=[2])
print(cpd_Y_on_X)

+------+------+------+
| X    | X(0) | X(1) |
+------+------+------+
| Y(0) | 0.95 | 0.01 |
+------+------+------+
| Y(1) | 0.05 | 0.99 |
+------+------+------+


In [4]:
# 사전확률, 가능도 모두 구했다. 
# 베이지안 모형에 넣자. 
from pgmpy.models import BayesianModel
model = BayesianModel([('X','Y')])
model.add_cpds(cpd_Y_on_X,cpd_X) # 사전확률모형 # 가능도모형 추가 
model.check_model() # 베이지안 모형 정상이다. 

True

In [6]:
# 베이지안 모형 이용한 추정 --> 사후확률 계산하기
from pgmpy.inference import VariableElimination

inference = VariableElimination(model)
posterior = inference.query(['X'],evidence={'Y':1})
print(posterior) # X=0(P(D|S)), X=1(P(DC|S))의 사후확률


0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]

0it [00:00, ?it/s]+------+----------+
| X    |   phi(X) |
+======+==========+
| X(0) |   0.9618 |
+------+----------+
| X(1) |   0.0382 |
+------+----------+



# 이진분류문제로 풀었으면 X=0으로 분류했을 것이다. (병에 걸리지 않았다!)

In [10]:
# 다시 패키지 사용법 연습하면서 이진분류문제 풀어보자. 
# 1. 사전확률모형 구하자. 
from pgmpy.factors.discrete import TabularCPD

cpd_X = TabularCPD('X',2,np.array([[0.998],[0.002]])) # 사전확률모형
print('사전확률모형')
print(cpd_X)

사전확률모형
+------+-------+
| X(0) | 0.998 |
+------+-------+
| X(1) | 0.002 |
+------+-------+


In [24]:
# 2. 가능도 모형
cpd_Y_on_X = TabularCPD('Y',2,np.array([[0.95,0.01],[0.05,0.99]]),
evidence=['X'],evidence_card=[2])
print('가능도모형')
print(cpd_Y_on_X)

가능도모형
+------+------+------+
| X    | X(0) | X(1) |
+------+------+------+
| Y(0) | 0.95 | 0.01 |
+------+------+------+
| Y(1) | 0.05 | 0.99 |
+------+------+------+


In [32]:
# 베이지안모형 생성
from pgmpy.models import BayesianModel
model = BayesianModel([('X','Y')])
model.add_cpds(cpd_X,cpd_Y_on_X)
model.check_model() # 베이지안 모형 생성 완료

True

In [39]:
from pgmpy.inference import VariableElimination
inference = VariableElimination(model) # 추정객체 생성
print(inference.query(['X'],evidence={'Y':1}))


0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]

0it [00:00, ?it/s]+------+----------+
| X    |   phi(X) |
+======+==========+
| X(0) |   0.9618 |
+------+----------+
| X(1) |   0.0382 |
+------+----------+



# 베이즈 정리의 확장 2
- P(A|B,C) = (P(C|A,B)P(A|B))/P(C|B)
- 사건 A에 대해 여러 사건 동시에 발생했을 때(B,C...) 사건 A 사후확률은?
- 증명 : [조건부확률-결합확률] 사슬법칙 두번 적용해서 증명한다. 

# 사슬법칙 사용하면 여러 종류 베이즈정리 확장 식 증명할 수 있다. 

# 몬티 홀 문제 정의하기 
- 핵심 : 문제가 결국 이진분류문제다. 진행자는 반드시 염소를 제거할 수 밖에 없고, 내 최종선택은 염소 vs 자동차 둘 중 하나에서 하게 된다. 
- 최종선택 : 내가 처음에 고른 문 뒤에 자동차가 있다 vs 내가 선택하지 않은 문에 자동차가 있다. - [결과가_둘_중_하나로_나오는_이진분류문제다]

따라서 나는 '내가 처음에 고른 문 뒤에 자동차가 있는 확률 vs '내가 선택하지 않은 문에 자동차가 있는 확률' 을 비교하면 된다. 

둘 중 확률이 높은 것이 채택되고, 결과값이 된다. 예) 이 상황은 내가 처음에 고른 문 뒤에 자동차가 있다! 고 분류될 것이다. 

한편, 내 선택의 관점에서 보면 둘 중 확률 높은 것을 선택하는 게 [자동차_당첨에_유리할_것이다]

---
# 문제해결 _1
- C : 자동차가 있는 경우를 의미하는 확률변수. 가질 수 있는 값 =(0,1,2)
- X : 내 선택한 문을 뜻하는 확률변수. 가질 수 있는 값 =(0,1,2)
- H : 진행자가 선택하는 문을 뜻하는 확률변수. 가질 수 있는 값 =(0,1,2)

유의점 1 : 확률변수 C와 확률변수 X는 서로 독립(=영향 미치지 않음)이다. 따라서 P(C,X) = P(C)P(X) 로 나타낼 수 있다.
유의점 2 : 진행자의 선택은 내가 어떤 문을 골랐느냐 & 자동차 위치가 어디였느냐에 달려있다. 따라서 진행자 선택을 뜻하는 조건부확률은 다음과 같다. 

1. 참가자 첫 선택이 자동차 아닌 경우 

P(H1|X2,C0) = 1
P(H0|X2,C0) = 0
P(H2|X2,C0) = 0

2. 참가자 첫 선택이 알고보니 자동차인 경우 

P(H0|X2,C2) = 1/2
P(H1|X2,C2) = 1/2
P(H2|X2,C2) = 0


# 문제해결 _2 
이제 참가자가 고른 문 뒤에 차가 있을 확률 vs 참가자가 고르지 않은 문 뒤에 차 있을 확률 비교하면 된다. 
두 확률 합은 1이다. (이진분류문제다. 예를들어 c1,c0둘 중 하나에 차가 있다면 결과는 c1에 차가 있다 vs c0에 차가 있다 둘 중 하나다. 확률 공리 이용해 교집합 없는 두 사건 합치면 합은 1이 된다)
식으로 쓰면 다음과 같다.

P(C1|X2,H1)+P(C0|X2,H1) = 1

이제 문제를 풀어보자. 
두 케이스 중 하나의 확률을 구하면 나머지 확률도 구할 수 있으므로, 나는 참가자가 첫번째 고른 문 뒤에 차가 있을 확률'을 구했다. 


P(C2|X2,H1) = P(C2,X2,H1)/P(X2,H1) = P(H1|C2,X2)P(C2,X2)/P(X2,H1)

= P(H1|C2,X2)P(C2)/P(H1|X2) = (1/2)P(C2)/(P(H1,C0|X2)+P(H1,C1|X2)+P(H1,C2|X2))

= ((1/2)(1/3))/(P(H1|C0,X2)P(C0)+P(H1|C1,X2)P(C1)+P(H1|C2,X2)P(C2))

= ((1/2)(1/3))/((1/3)+(1/2)(1/3))

= [1/3]

따라서 내가 처음에 골랐던 문 뒤에 자동차가 있을 확률은 1/3이다. 

1-(1/3)하면 진행자가 열지 않았고, 내가 선택하지 않았던 남은 문 하나에 자동차 있을 확률이 2/3으로 나온다. 

2/3은 1/3의 두배이므로, 내가 고르지 않은 문 뒤에 결과적으로 자동차가 있을 확률이 더 높다. 

따라서 '자동차 당첨을 위해서는' 선택을 바꾸는 것이 ['유리하다']



